In [ ]:
#Receiving data from OpenBCI GUI TEST
import argparse
import time
import asyncio
import numpy as np
import pandas as pd
from pythonosc import dispatcher
from pythonosc import osc_server
from pythonosc import udp_client
from pythonosc import osc_message_builder
from pythonosc import osc_bundle_builder
from typing import List, Any

from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPClassifier

#Reading and joining both datasets
dataset=pd.read_csv('Dataset1_3SecJoinedFT_Flat.csv',header=None).values
ds2=pd.read_csv('Dataset2_3SecJoinedFT_FlatOS.csv',header=None).values
dataset=np.append(dataset,ds2,axis=0)
x=dataset[:,:-2]

y=dataset[:,-2:]


y1=y[:,0].astype(int)
y2=y[:,1].astype(int)


#Train classifiers (MLP and KNN)
print('Training Classifiers...')
mlr1=MLPClassifier(solver='adam',activation='relu',alpha=1e-6,hidden_layer_sizes=(600,),max_iter=2000, random_state=0)
mlr2=MLPClassifier(solver='adam',activation='relu',alpha=1e-6,hidden_layer_sizes=(600,),max_iter=2000, random_state=0)
mlr1.fit(x,y1)
mlr2.fit(x,y2)

knn1=KNeighborsRegressor(n_neighbors=5)
knn2=KNeighborsRegressor(n_neighbors=5)
knn1.fit(x,y1)
knn2.fit(x,y2)

print('Done!')

#Intialize list of Timeseries values
vals=[]
def print_test(addr,*args:List[Any]):
    vals.append(np.asarray(args)[2:])

def classify(using):
    arr=np.zeros((14,75))
    for col in range(14):
        fft=(np.abs(np.fft.hfft(using[:,col]))).tolist()
        fft=np.asarray(fft[1:76]) #1-25 hz
        arr[col]=fft
    arr=arr.flatten()
    val=mlr1.predict(arr.reshape(1,-1))[0]
    aro=mlr2.predict(arr.reshape(1,-1))[0]
    print("\n\nValence: MLP",val,"\tKNN",knn1.predict(arr.reshape(1,-1))[0])
    print("Arousal: MLP",aro,"\tKNN",knn2.predict(arr.reshape(1,-1))[0])
    return val,aro
    
    
dispatcher = dispatcher.Dispatcher()
dispatcher.map("/openbci", print_test)
unity=udp_client.SimpleUDPClient("127.0.0.1", 5555)
    
async def loop():
    while True:
        curr=np.asarray(vals)
        if (curr.shape[0]>=375):
            curr=curr[-375:]
            valence,arousal=classify(curr)
            unity.send_message("/python",[float(valence),float(arousal)])
            
        await asyncio.sleep(1)


async def init_main():
    server = osc_server.AsyncIOOSCUDPServer(('127.0.0.1', 12347), dispatcher, asyncio.get_event_loop())
    transport, protocol = await server.create_serve_endpoint()  # Create datagram endpoint and start serving

    await loop()  # Enter main loop of program

    transport.close()  # Clean up serve endpoint


await init_main()